In [3]:
# Install libraries I need for my project
# I'm using Plotly for charts, ipywidgets for the form, and scikit-learn for predictions
!pip install --upgrade plotly ipywidgets scikit-learn
from google.colab import output
# Enable widgets to make my form interactive in Colab
output.enable_custom_widget_manager()

# Import libraries for the project
import sqlite3  # To store sales data
import pandas as pd  # To handle data in tables
import plotly.express as px  # To create charts
import ipywidgets as widgets  # To build the input form
from IPython.display import display, clear_output  # To show the form and clear output
from sklearn.linear_model import LinearRegression  # To predict sales
import numpy as np  # For calculations
from datetime import datetime  # For dates

# Set up the database to store sales data
def init_db():
    try:
        # Connect to my database file
        conn = sqlite3.connect('sales.db')
        c = conn.cursor()
        # Create a table called 'sales' to store data
        c.execute('''CREATE TABLE IF NOT EXISTS sales
                     (id INTEGER PRIMARY KEY AUTOINCREMENT, product TEXT, quantity INTEGER, price FLOAT, season TEXT, month TEXT, predicted_sales FLOAT)''')
        # Add sample data if the table is empty
        c.execute("SELECT COUNT(*) FROM sales")
        if c.fetchone()[0] == 0:
            # Sample data I created in INR for testing
            sample_data = [
                ('Laptop', 50, 50000.0, 'Summer', '2025-06', 2500000.0),
                ('Phone', 30, 30000.0, 'Winter', '2025-01', 900000.0),
                ('Tablet', 20, 20000.0, 'Spring', '2025-03', 400000.0),
                ('Headphones', 40, 5000.0, 'Fall', '2025-09', 200000.0)
            ]
            c.executemany("INSERT INTO sales (product, quantity, price, season, month, predicted_sales) VALUES (?, ?, ?, ?, ?, ?)", sample_data)
        conn.commit()
        conn.close()
        print("Database ready with sample sales data in INR.")
    except Exception as e:
        print(f"Oops, error setting up database: {e}")

# Train a simple machine learning model to predict sales
def train_model():
    # I used a small dataset in INR to learn ML
    X = np.array([[50, 50000, 1], [30, 30000, 2], [20, 20000, 3], [40, 5000, 4]])  # Features: quantity, price, season
    y = np.array([2500000, 900000, 400000, 200000])  # Sales in INR
    model = LinearRegression()  # Simple model for predictions
    model.fit(X, y)
    return model

# Add a sale and predict next month's sales
def add_sale(product, quantity, price, season, month, model):
    try:
        # Convert season to a number for the model
        season_map = {'Summer': 1, 'Winter': 2, 'Spring': 3, 'Fall': 4}
        features = np.array([[quantity, price, season_map[season]]])
        # Predict sales and round to 2 decimals
        predicted_sales = round(model.predict(features)[0], 2)
        # Keep predictions between ₹50,000 and ₹5,000,000
        predicted_sales = max(50000, min(5000000, predicted_sales))
        # Save to database
        conn = sqlite3.connect('sales.db')
        c = conn.cursor()
        c.execute("INSERT INTO sales (product, quantity, price, season, month, predicted_sales) VALUES (?, ?, ?, ?, ?, ?)",
                  (product, quantity, price, season, month, predicted_sales))
        conn.commit()
        conn.close()
        print(f"Added: {product}, {quantity} units, ₹{price:,.2f}, {season}, {month}, Predicted Sales: ₹{predicted_sales:,.2f}")
        return predicted_sales
    except Exception as e:
        print(f"Error adding sale: {e}")
        return None

# Load and show sales data
def analyze_sales():
    try:
        conn = sqlite3.connect('sales.db')
        # Load data into a table
        df = pd.read_sql_query("SELECT * FROM sales", conn)
        conn.close()
        if not df.empty:
            print("My Sales Data (in INR):")
            print(df[['product', 'quantity', 'price', 'season', 'month', 'predicted_sales']])
        return df
    except Exception as e:
        print(f"Error loading sales: {e}")
        return pd.DataFrame()

# Show charts for sales trends
def show_visualizations():
    df = analyze_sales()
    if df.empty:
        print("No sales data to show yet!")
        return

    try:
        # Convert month for plotting
        df['month'] = pd.to_datetime(df['month'], format='%Y-%m')
        # Line chart for sales over time
        monthly_sales = df.groupby('month')['predicted_sales'].sum().reset_index()
        fig_line = px.line(monthly_sales, x='month', y='predicted_sales', title='Predicted Sales Over Time (INR)',
                           labels={'predicted_sales': 'Predicted Sales (₹)', 'month': 'Month'})
        fig_line.update_layout(width=600, height=400)
        fig_line.show()

        # Bar chart for average sales by season
        season_avg = df.groupby('season')['predicted_sales'].mean().reset_index()
        fig_bar = px.bar(season_avg, x='season', y='predicted_sales', title='Average Predicted Sales by Season (INR)',
                         labels={'predicted_sales': 'Average Sales (₹)'}, color='season',
                         color_discrete_sequence=px.colors.qualitative.Plotly)
        fig_bar.update_layout(width=600, height=400)
        fig_bar.show()

        # Show stats
        print(f"Total Sales Entries: {len(df)}")
        print(f"Average Predicted Sales: ₹{df['predicted_sales'].mean():,.2f}")
    except Exception as e:
        print(f"Error showing charts: {e}")

# Create a form for users to add sales
product_input = widgets.Dropdown(options=['Laptop', 'Phone', 'Tablet', 'Headphones'], description="Product:")
quantity_input = widgets.IntSlider(min=1, max=100, step=1, value=10, description="Quantity:")
price_input = widgets.FloatSlider(min=1000, max=100000, step=500, value=50000, description="Price (₹):")
season_input = widgets.Dropdown(options=['Summer', 'Winter', 'Spring', 'Fall'], description="Season:")
month_input = widgets.DatePicker(description="Month:", value=datetime.now())
submit_button = widgets.Button(description="Add Sale", button_style='success')
viz_button = widgets.Button(description="Show Visualizations", button_style='info')

# Train the model
model = train_model()

# Handle form submissions
def on_submit_clicked(b):
    clear_output()
    display(product_input, quantity_input, price_input, season_input, month_input, submit_button, viz_button)
    if product_input.value and quantity_input.value and price_input.value and season_input.value and month_input.value:
        month_str = month_input.value.strftime('%Y-%m')
        add_sale(product_input.value, quantity_input.value, price_input.value, season_input.value, month_str, model)
    else:
        print("Please fill all fields!")

# Handle visualization button
def on_viz_clicked(b):
    clear_output()
    display(product_input, quantity_input, price_input, season_input, month_input, submit_button, viz_button)
    show_visualizations()

submit_button.on_click(on_submit_clicked)
viz_button.on_click(on_viz_clicked)

# Start the database and show the form
init_db()
display(product_input, quantity_input, price_input, season_input, month_input, submit_button, viz_button)

Dropdown(description='Product:', index=3, options=('Laptop', 'Phone', 'Tablet', 'Headphones'), value='Headphon…

IntSlider(value=60, description='Quantity:', min=1)

FloatSlider(value=37000.00000000001, description='Price (₹):', max=100000.0, min=1000.0, step=500.0)

Dropdown(description='Season:', index=3, options=('Summer', 'Winter', 'Spring', 'Fall'), value='Fall')

DatePicker(value=datetime.date(2024, 5, 12), description='Month:', step=1)

Button(button_style='success', description='Add Sale', style=ButtonStyle())

Button(button_style='info', description='Show Visualizations', style=ButtonStyle())

My Sales Data (in INR):
       product  quantity    price  season    month  predicted_sales
0       Laptop        50    800.0  Summer  2025-06         40000.00
1        Phone        30    600.0  Winter  2025-01         18000.00
2       Tablet        20    400.0  Spring  2025-03          8000.00
3       Laptop        40    850.0    Fall  2025-09         34000.00
4       Laptop        10    500.0  Summer  2025-06          1000.00
5   Headphones        62    530.0  Summer  2025-06         54400.00
6   Headphones        26    530.0  Summer  2025-06         11200.00
7   Headphones        50    530.0  Summer  2025-03         40000.00
8       Laptop        34  50000.0  Summer  2025-06       2111428.57
9       Laptop        34  37000.0  Spring  2025-05       2197142.86
10  Headphones        60  37000.0    Fall  2024-05       3428571.43
11  Headphones        60  37000.0    Fall  2024-05       3428571.43


Total Sales Entries: 12
Average Predicted Sales: ₹947,692.86
